In [1]:
# This is file for train, prediction

import pandas as pd

dirpath = 'C:/Users/rihot/Desktop/Deep_learning/DACON_used_car_price/'

train = pd.read_csv('data/preprocessed_train.csv')
test = pd.read_csv('data/preprocessed_test.csv')

In [2]:
Y = train[['target']].values
X = train[ ['title', 'odometer', 'location', 'isimported', 'engine', 'paint', 'year', 'brand' ] ].values

X_test = test[ ['title', 'odometer', 'location', 'isimported', 'engine', 'paint', 'year', 'brand' ] ].values

# X = train[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values
#
# X_test = test[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values

In [3]:
X.shape, Y.shape

((1015, 8), (1015, 1))

In [4]:
from sklearn.preprocessing import MinMaxScaler

scalerX = MinMaxScaler()
scalerX.fit(X)
X = scalerX.transform(X)
X_test = scalerX.transform(X_test)

scalerY = MinMaxScaler()
scalerY.fit(Y)
Y = scalerY.transform(Y)

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna
from optuna import Trial
from lightgbm import LGBMRegressor
import xgboost as xgb

def LGBM(trial : Trial, X, Y, test):
    param = {
        'objective': 'regression', # 회귀
        'metric': 'rmse',
        "n_estimators" : trial.suggest_categorical('n_estimators', [500, 600,700,800,900,1000,1200,1400,1600,1800,2000, 4000]),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01,0.02,0.05]),
        # 'nthread' : -1,
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = LGBMRegressor(**param)
    LGBM_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(LGBM_model.predict(X_test), y_test, squared=False)

    return score

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import numpy as np
from optuna.samplers import TPESampler

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : LGBM(trial, X,  Y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-02 23:55:31,269] A new study created in memory with name: no-name-c5b49c00-f899-479e-a6e2-1049aab8bae2
[I 2022-06-02 23:55:33,461] Trial 0 finished with value: 0.05465324373714255 and parameters: {'n_estimators': 4000, 'max_depth': 14, 'learning_rate': 0.02, 'subsample': 1.0}. Best is trial 0 with value: 0.05465324373714255.
[I 2022-06-02 23:55:33,982] Trial 1 finished with value: 0.034577816643479066 and parameters: {'n_estimators': 1400, 'max_depth': 8, 'learning_rate': 0.02, 'subsample': 0.7}. Best is trial 1 with value: 0.034577816643479066.
[I 2022-06-02 23:55:34,260] Trial 2 finished with value: 0.04663499004303781 and parameters: {'n_estimators': 600, 'max_depth': 9, 'learning_rate': 0.01, 'subsample': 0.6}. Best is trial 1 with value: 0.034577816643479066.
[I 2022-06-02 23:55:34,671] Trial 3 finished with value: 0.04226636350908355 and parameters: {'n_estimators': 700, 'max_depth': 13, 'learning_rate': 0.05, 'subsample': 0.6}. Best is trial 1 with value: 0.0345778166

Best trial: score 0.02670091692246342,
params {'n_estimators': 2000, 'max_depth': 9, 'learning_rate': 0.02, 'subsample': 0.8}


Best trial: score 0.02950600044760606,
params {'n_estimators': 500, 'max_depth': 12, 'learning_rate': 0.02, 'subsample': 0.8}

In [9]:
from xgboost import XGBRegressor

def XGB(trial : Trial, X, Y, test):
    param = {
        # 'objective': 'regression', # 회귀
        'metric': 'rmse',
        "n_estimators" : trial.suggest_categorical('n_estimators', [500, 600,700,800,900,1000,1200,1400,1600,1800,2000, 4000]),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01,0.02,0.05]),
        # 'nthread' : -1,
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = XGBRegressor(**param)
    XGB_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(XGB_model.predict(X_test), y_test, squared=False)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : XGB(trial, X,  Y, X_test), n_trials=50)

print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-03 00:01:19,042] A new study created in memory with name: no-name-333f7c87-1662-4968-b267-896f708fb965


[00:01:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-06-03 00:01:19,926] Trial 0 finished with value: 0.0839424345688986 and parameters: {'n_estimators': 1800, 'max_depth': 8, 'learning_rate': 0.05, 'subsample': 0.6}. Best is trial 0 with value: 0.0839424345688986.


[00:01:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-06-03 00:01:20,592] Trial 1 finished with value: 0.05331714185330453 and parameters: {'n_estimators': 800, 'max_depth': 15, 'learning_rate': 0.01, 'subsample': 0.6}. Best is trial 1 with value: 0.05331714185330453.


...

[00:01:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-06-03 00:01:54,608] Trial 48 finished with value: 0.0492064880653608 and parameters: {'n_estimators': 1400, 'max_depth': 10, 'learning_rate': 0.02, 'subsample': 1.0}. Best is trial 4 with value: 0.02088880864021991.


[00:01:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "metric" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




[I 2022-06-03 00:01:55,180] Trial 49 finished with value: 0.08382543566422782 and parameters: {'n_estimators': 700, 'max_depth': 13, 'learning_rate': 0.01, 'subsample': 0.8}. Best is trial 4 with value: 0.02088880864021991.


Best trial: score 0.02088880864021991,
params {'n_estimators': 900, 'max_depth': 16, 'learning_rate': 0.02, 'subsample': 1.0}


Best trial: score 0.02088880864021991,
params {'n_estimators': 900, 'max_depth': 16, 'learning_rate': 0.02, 'subsample': 1.0}

In [12]:
from catboost import CatBoostRegressor

def cat(trial : Trial, X, Y, test):
    param = {
        # 'objective': 'regression', # 회귀
        # 'metric': 'rmse',
        "n_estimators" : trial.suggest_categorical('n_estimators', [500, 600,700,800,900,1000,1200,1400,1600,1800,2000, 4000]),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01,0.02,0.05]),
        # 'nthread' : -1,
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = CatBoostRegressor(**param)
    cat_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(cat_model.predict(X_test), y_test, squared=False)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : cat(trial, X,  Y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-03 00:03:17,052] A new study created in memory with name: no-name-7f36e082-a840-4fc4-bc43-d795703688ff
[I 2022-06-03 00:03:56,946] Trial 0 finished with value: 0.06047475812189175 and parameters: {'n_estimators': 1400, 'max_depth': 12, 'learning_rate': 0.02, 'subsample': 0.6}. Best is trial 0 with value: 0.06047475812189175.
[I 2022-06-03 00:04:03,142] Trial 1 finished with value: 0.05307147628981088 and parameters: {'n_estimators': 800, 'max_depth': 10, 'learning_rate': 0.02, 'subsample': 1.0}. Best is trial 1 with value: 0.05307147628981088.
[I 2022-06-03 00:04:20,953] Trial 2 finished with value: 0.044453107955052716 and parameters: {'n_estimators': 1200, 'max_depth': 11, 'learning_rate': 0.02, 'subsample': 0.8}. Best is trial 2 with value: 0.044453107955052716.
[I 2022-06-03 00:04:24,641] Trial 3 finished with value: 0.04991663524519707 and parameters: {'n_estimators': 1600, 'max_depth': 8, 'learning_rate': 0.05, 'subsample': 0.6}. Best is trial 2 with value: 0.044453107

Best trial: score 0.02228076617330138,
params {'n_estimators': 800, 'max_depth': 10, 'learning_rate': 0.02, 'subsample': 0.8}


Best trial: score 0.02228076617330138,
params {'n_estimators': 800, 'max_depth': 10, 'learning_rate': 0.02, 'subsample': 0.8}

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

def gradi(trial : Trial, X, Y, test):
    param = {
        # 'objective': 'regression', # 회귀
        # 'metric': 'rmse',
        "n_estimators" : trial.suggest_categorical('n_estimators', [500, 600,700,800,900,1000,1200,1400,1600,1800,2000, 4000]),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01,0.02,0.05]),
        # 'nthread' : -1,
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = GradientBoostingRegressor(**param)
    gradi_model = model.fit(X_train, y_train)
    score = mean_squared_error(gradi_model.predict(X_test), y_test, squared=False)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : gradi(trial, X,  Y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-03 01:55:12,903] A new study created in memory with name: no-name-618a3711-610d-442b-a1d2-24f39ab1fea4
[I 2022-06-03 01:55:15,419] Trial 0 finished with value: 0.05914506379597146 and parameters: {'n_estimators': 1200, 'max_depth': 12, 'learning_rate': 0.02, 'subsample': 0.8}. Best is trial 0 with value: 0.05914506379597146.
[I 2022-06-03 01:55:16,592] Trial 1 finished with value: 0.03949706502543835 and parameters: {'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.02, 'subsample': 0.6}. Best is trial 1 with value: 0.03949706502543835.
[I 2022-06-03 01:55:19,545] Trial 2 finished with value: 0.05288881995786198 and parameters: {'n_estimators': 1400, 'max_depth': 11, 'learning_rate': 0.05, 'subsample': 1.0}. Best is trial 1 with value: 0.03949706502543835.
[I 2022-06-03 01:55:21,107] Trial 3 finished with value: 0.08332525123170709 and parameters: {'n_estimators': 800, 'max_depth': 13, 'learning_rate': 0.05, 'subsample': 0.7}. Best is trial 1 with value: 0.039497065025

Best trial: score 0.02844488766002739,
params {'n_estimators': 700, 'max_depth': 9, 'learning_rate': 0.05, 'subsample': 0.8}


Best trial: score 0.02844488766002739,
params {'n_estimators': 700, 'max_depth': 9, 'learning_rate': 0.05, 'subsample': 0.8}

In [26]:
from sklearn.ensemble import ExtraTreesRegressor

def etr(trial : Trial, X, Y, test):
    param = {
        # 'objective': 'regression', # 회귀
        # 'metric': 'rmse',
        "n_estimators" : trial.suggest_categorical('n_estimators', [500, 600,700,800,900,1000,1200,1400,1600,1800,2000, 4000]),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        # 'learning_rate': trial.suggest_categorical('learning_rate', [0.01,0.02,0.05]),
        # 'nthread' : -1,
        # 'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = ExtraTreesRegressor(**param)
    etr_model = model.fit(X_train, y_train)
    score = mean_squared_error(etr_model.predict(X_test), y_test, squared=False)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : etr(trial, X,  Y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-03 01:58:00,537] A new study created in memory with name: no-name-dd0b8811-9325-4723-8262-e0e3fe5213ca
[I 2022-06-03 01:58:08,097] Trial 0 finished with value: 0.09096298022115129 and parameters: {'n_estimators': 4000, 'max_depth': 15}. Best is trial 0 with value: 0.09096298022115129.
[I 2022-06-03 01:58:09,217] Trial 1 finished with value: 0.02877347975814291 and parameters: {'n_estimators': 600, 'max_depth': 15}. Best is trial 1 with value: 0.02877347975814291.
[I 2022-06-03 01:58:15,637] Trial 2 finished with value: 0.051236962110817054 and parameters: {'n_estimators': 4000, 'max_depth': 13}. Best is trial 1 with value: 0.02877347975814291.
[I 2022-06-03 01:58:17,146] Trial 3 finished with value: 0.045068519397612514 and parameters: {'n_estimators': 700, 'max_depth': 15}. Best is trial 1 with value: 0.02877347975814291.
[I 2022-06-03 01:58:18,451] Trial 4 finished with value: 0.05819010410208789 and parameters: {'n_estimators': 700, 'max_depth': 13}. Best is trial 1 with 

Best trial: score 0.02668045501392634,
params {'n_estimators': 600, 'max_depth': 9}


Best trial: score 0.02668045501392634,
params {'n_estimators': 600, 'max_depth': 9}

In [32]:
from sklearn.ensemble import RandomForestRegressor

def rfr(trial : Trial, X, Y, test):
    param = {
        # 'objective': 'regression', # 회귀
        # 'metric': 'rmse',
        "n_estimators" : trial.suggest_categorical('n_estimators', [500, 600,700,800,900,1000,1200,1400,1600,1800,2000, 4000]),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        # 'learning_rate': trial.suggest_categorical('learning_rate', [0.01,0.02,0.05]),
        # 'nthread' : -1,
        # 'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = RandomForestRegressor(**param)
    rfr_model = model.fit(X_train, y_train)
    score = mean_squared_error(rfr_model.predict(X_test), y_test, squared=False)

    return score

study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : rfr(trial, X,  Y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

[I 2022-06-03 02:15:57,377] A new study created in memory with name: no-name-8870f84b-4218-4298-b11d-01d21a3ace53
[I 2022-06-03 02:15:58,548] Trial 0 finished with value: 0.04131496640430345 and parameters: {'n_estimators': 600, 'max_depth': 8}. Best is trial 0 with value: 0.04131496640430345.
[I 2022-06-03 02:16:08,172] Trial 1 finished with value: 0.03484342822735787 and parameters: {'n_estimators': 4000, 'max_depth': 12}. Best is trial 1 with value: 0.03484342822735787.
[I 2022-06-03 02:16:09,348] Trial 2 finished with value: 0.0805269800774792 and parameters: {'n_estimators': 500, 'max_depth': 12}. Best is trial 1 with value: 0.03484342822735787.
[I 2022-06-03 02:16:13,447] Trial 3 finished with value: 0.0884373574909707 and parameters: {'n_estimators': 1600, 'max_depth': 14}. Best is trial 1 with value: 0.03484342822735787.
[I 2022-06-03 02:16:15,792] Trial 4 finished with value: 0.03472087963963781 and parameters: {'n_estimators': 900, 'max_depth': 15}. Best is trial 4 with value

Best trial: score 0.033730265932558116,
params {'n_estimators': 2000, 'max_depth': 10}


Best trial: score 0.033730265932558116,
params {'n_estimators': 2000, 'max_depth': 10}